In [1]:
from getpass import getpass
from typing import List, Optional

import requests
from huggingface_hub import HfApi
from huggingface_hub.hf_api import HfApi, HfFolder
from pydantic import BaseModel, IPvAnyAddress, validator
from requests.exceptions import HTTPError

In [2]:
ENDPOINT_WHITELIST = "https://collaborative-training-auth.huggingface.co"


class HFTokenManagemment:
    def __init__(self):
        self._api = HfApi()

    def login(self):
        username = input("Experiment owner username: ")
        password = getpass()
        try:
            token = self._api.login(username, password)
        except HTTPError as e:
            # probably invalid credentials, display error message.
            print(e)
            print(e.response.text)
            exit(1)
        HfFolder.save_token(token)
        print("Login successful")
        print("Your token:", token, "\n")
        print("Your token has been saved to", HfFolder.path_token)

    def logout(self):
        token = HfFolder.get_token()
        if token is None:
            print("Not logged in")
            exit()
        HfFolder.delete_token()
        self._api.logout(token)
        print("Successfully logged out.")

In [3]:
# Define models to help request to API
class UserCreate(BaseModel):
    """
    username are required for registering a new user
    """

    username: str


class ExperimentFullCreatePublic(BaseModel):
    name: Optional[str]
    coordinator_ip: Optional[IPvAnyAddress]
    coordinator_port: Optional[int]
    auth_server_public_key: Optional[bytes]
    auth_server_private_key: Optional[bytes]
    collaborators: Optional[List[UserCreate]]

    @validator("coordinator_port")
    def validate_port(cls, port):
        if port is None:
            return port

        if int(port) > 2 ** 16:
            raise ValueError("port overflow")
        return port


class ExperimentFullUpdate(BaseModel):
    name: Optional[str]
    coordinator_ip: Optional[IPvAnyAddress]
    coordinator_port: Optional[int]
    added_collaborators: Optional[List[UserCreate]]
    removed_collaborators: Optional[List[UserCreate]]

    @validator("coordinator_port")
    def validate_port(cls, port):
        if port is None:
            return port

        if int(port) > 2 ** 16:
            raise ValueError("port overflow")
        return port

In [4]:
# As a HF user, you ask a HF token session
token_management = HFTokenManagemment()
token_management.login()


        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
        
Username: Aline
········
Login successful
Your token: PfUCgGLjcJHktPfAJHLfabFhLwlWGLAfmZDquyGoPiqRfcSRstSQEwYbmIGiOpqThGrmAQNiyMbAEKcHgrLzkXcPqfQTdHAsVjNHvUmmzjSonmiHsRveVpGIsgBRHalT 

Your token has been saved to /home/lucile/.huggingface/token


# Create a new experiment

In [ ]:
# This is the content request for the new experiment to create
new_experiment = ExperimentFullCreatePublic(
    name="fake experiment 1",
    collaborators=[UserCreate(username="borzunov")],
)

In [ ]:
# Send request to server
path = "{}/api/experiments/".format(ENDPOINT_WHITELIST)

headers = {"Authorization": f"Bearer {HfFolder.get_token()}"}
r = requests.post(path, headers=headers, json={"new_experiment": new_experiment.dict()})
r.raise_for_status()
d = r.json()

In [ ]:
# Show server response
d

# Update the experiment just created

In [ ]:
# This is the content request for the new experiment to create
experiment_full_update = ExperimentFullUpdate(
    name="fake experiment 1",
    added_collaborators=[UserCreate(username="Aline")],
)
id_experiment = 1

In [ ]:
# Send request to server
path = f"{ENDPOINT_WHITELIST}/api/experiments/{id_experiment}/"

headers = {"Authorization": f"Bearer {HfFolder.get_token()}"}
r = requests.put(
    path,
    headers=headers,
    json={"experiment_full_update": experiment_full_update.dict()},
)
r.raise_for_status()
d = r.json()

In [ ]:
# Show server response
d

In [ ]:
# As a HF user, you terminate your session (your token is not longer valid)
token_management.logout()